In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Kidney-Disease-Classification-MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Kidney-Disease-Classification-MLOPS'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmented: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmented=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [17]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerators=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerators.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmented:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerators

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [18]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2026-01-07 11:42:28,483]: INFO: common: yaml file: config\config.yaml loaded successfully
[2026-01-07 11:42:28,487]: INFO: common: yaml file: params.yaml loaded successfully
[2026-01-07 11:42:28,489]: INFO: common: created directory at: artifacts
[2026-01-07 11:42:28,491]: INFO: common: created directory at: artifacts\training
Found 8 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Epoch 1/10
6/6 [==============================] - 7s 643ms/step - loss: 2.6223 - accuracy: 0.2593 - val_loss: 1.5578 - val_accuracy: 0.2000
Epoch 2/10
6/6 [==============================] - 1s 134ms/step - loss: 1.6348 - accuracy: 0.2963 - val_loss: 1.1763 - val_accuracy: 0.4000
Epoch 3/10
6/6 [==============================] - 1s 126ms/step - loss: 1.7614 - accuracy: 0.3333 - val_loss: 1.1425 - val_accuracy: 0.6000
Epoch 4/10
6/6 [==============================] - 1s 123ms/step - loss: 1.5033 - accuracy: 0.4074 - val_loss: 1.6100 - val_accuracy: 0.4000
Epoch 5/10
6/6 [===============